<a href="https://colab.research.google.com/github/gremlin97/EVA-8/blob/main/S4/Part2/Kunal_Eva3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [22]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #3
        self.bn1 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2) #6
        self.drop1 = nn.Dropout(0.15)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=1) #8
        self.bn2 = nn.BatchNorm2d(32)
        self.pool2 = nn.MaxPool2d(2, 2) #16
        self.drop2 = nn.Dropout(0.15)
        self.conv5 = nn.Conv2d(32, 32, 3) #18

        self.bn3 = nn.BatchNorm2d(32) 
        self.drop3 = nn.Dropout(0.15)
        self.conv6 = nn.Conv2d(32, 10, 3) 

        self.gap = nn.AdaptiveAvgPool2d((1,1))

    def forward(self, x):

        x = self.conv6(self.drop3(self.bn3(F.relu(self.conv5(self.drop2(self.pool2(self.bn2(F.relu(self.conv3(F.relu(self.drop1(self.pool1(self.bn1(F.relu(self.conv1(x))))))))))))))))

        x = self.gap(x)
        
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [23]:
model = Net()
out = model(torch.randn(1,1,28,28))
print(out.shape)

torch.Size([1, 10])


<ipython-input-22-e8be6f7f95f0>:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [24]:
model

Net(
  (conv1): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop1): Dropout(p=0.15, inplace=False)
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (pool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (drop2): Dropout(p=0.15, inplace=False)
  (conv5): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1))
  (bn3): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (drop3): Dropout(p=0.15, inplace=False)
  (conv6): Conv2d(32, 10, kernel_size=(3, 3), stride=(1, 1))
  (gap): AdaptiveAvgPool2d(output_size=(1, 1))
)

In [25]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
         MaxPool2d-3           [-1, 16, 14, 14]               0
           Dropout-4           [-1, 16, 14, 14]               0
            Conv2d-5           [-1, 32, 14, 14]           4,640
       BatchNorm2d-6           [-1, 32, 14, 14]              64
         MaxPool2d-7             [-1, 32, 7, 7]               0
           Dropout-8             [-1, 32, 7, 7]               0
            Conv2d-9             [-1, 32, 5, 5]           9,248
      BatchNorm2d-10             [-1, 32, 5, 5]              64
          Dropout-11             [-1, 32, 5, 5]               0
           Conv2d-12             [-1, 10, 3, 3]           2,890
Adap

<ipython-input-22-e8be6f7f95f0>:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [26]:
torch.manual_seed(1)
batch_size = 32

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [27]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [30]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.012, momentum=0.9)

#lr=0.01

for epoch in range(18):
    print("Epoch: ",epoch+1)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

Epoch:  1


  0%|          | 0/1875 [00:00<?, ?it/s]<ipython-input-22-e8be6f7f95f0>:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.04736865311861038 batch_id=1874: 100%|██████████| 1875/1875 [00:27<00:00, 67.42it/s]



Test set: Average loss: 0.0382, Accuracy: 9878/10000 (99%)

Epoch:  2


loss=0.12485630810260773 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 70.35it/s]



Test set: Average loss: 0.0319, Accuracy: 9894/10000 (99%)

Epoch:  3


...


Test set: Average loss: 0.0235, Accuracy: 9930/10000 (99%)

Epoch:  17


loss=0.0012909744400531054 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 71.14it/s]



Test set: Average loss: 0.0168, Accuracy: 9947/10000 (99%)

Epoch:  18


loss=0.002956618554890156 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 70.20it/s]



Test set: Average loss: 0.0190, Accuracy: 9940/10000 (99%)



In [29]:
# Maximum Accuracy Training Logs:

'''
Epoch:  13
loss=0.003916250541806221 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 70.80it/s]

Test set: Average loss: 0.0175, Accuracy: 9948/10000 (99%)

'''

# Training Logs:

'''
Epoch:  1
  0%|          | 0/1875 [00:00<?, ?it/s]<ipython-input-22-e8be6f7f95f0>:27: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.04736865311861038 batch_id=1874: 100%|██████████| 1875/1875 [00:27<00:00, 67.42it/s]

Test set: Average loss: 0.0382, Accuracy: 9878/10000 (99%)

Epoch:  2
loss=0.12485630810260773 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 70.35it/s]

Test set: Average loss: 0.0319, Accuracy: 9894/10000 (99%)

Epoch:  3
loss=0.00622600968927145 batch_id=1874: 100%|██████████| 1875/1875 [00:27<00:00, 68.48it/s]

Test set: Average loss: 0.0272, Accuracy: 9909/10000 (99%)

Epoch:  4
loss=0.056115299463272095 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 70.78it/s]

Test set: Average loss: 0.0231, Accuracy: 9926/10000 (99%)

Epoch:  5
loss=0.026963435113430023 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 70.55it/s]

Test set: Average loss: 0.0277, Accuracy: 9894/10000 (99%)

Epoch:  6
loss=0.0020414951723068953 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 70.38it/s]

Test set: Average loss: 0.0222, Accuracy: 9927/10000 (99%)

Epoch:  7
loss=0.0024074241518974304 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 70.80it/s]

Test set: Average loss: 0.0197, Accuracy: 9933/10000 (99%)

Epoch:  8
loss=0.018343215808272362 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 69.68it/s]

Test set: Average loss: 0.0214, Accuracy: 9931/10000 (99%)

Epoch:  9
loss=0.024688148871064186 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 70.49it/s]

Test set: Average loss: 0.0258, Accuracy: 9922/10000 (99%)

Epoch:  10
loss=0.018841002136468887 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 70.33it/s]

Test set: Average loss: 0.0190, Accuracy: 9938/10000 (99%)

Epoch:  11
loss=0.016844671219587326 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 70.29it/s]

Test set: Average loss: 0.0199, Accuracy: 9941/10000 (99%)

Epoch:  12
loss=0.0022426238283514977 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 70.46it/s]

Test set: Average loss: 0.0209, Accuracy: 9939/10000 (99%)

Epoch:  13
loss=0.003916250541806221 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 70.80it/s]

Test set: Average loss: 0.0175, Accuracy: 9948/10000 (99%)

Epoch:  14
loss=0.005738690495491028 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 69.85it/s]

Test set: Average loss: 0.0196, Accuracy: 9939/10000 (99%)

Epoch:  15
loss=0.004773998167365789 batch_id=1874: 100%|██████████| 1875/1875 [00:27<00:00, 67.55it/s]

Test set: Average loss: 0.0193, Accuracy: 9941/10000 (99%)

Epoch:  16
loss=0.008134204894304276 batch_id=1874: 100%|██████████| 1875/1875 [00:27<00:00, 67.96it/s]

Test set: Average loss: 0.0235, Accuracy: 9930/10000 (99%)

Epoch:  17
loss=0.0012909744400531054 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 71.14it/s]

Test set: Average loss: 0.0168, Accuracy: 9947/10000 (99%)

Epoch:  18
loss=0.002956618554890156 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 70.20it/s]

Test set: Average loss: 0.0190, Accuracy: 9940/10000 (99%)

'''

'\nEpoch:  14\nloss=0.0014290008693933487 batch_id=1874: 100%|██████████| 1875/1875 [00:26<00:00, 70.63it/s]\n\nTest set: Average loss: 0.0195, Accuracy: 9936/10000 (99%)\n\n'

In [31]:
# OLD MODEL BACKUP
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #3
#         self.bn1 = nn.BatchNorm2d(16)
#         # self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
#         self.pool1 = nn.MaxPool2d(2, 2) #6
#         self.drop1 = nn.Dropout(0.15)
#         self.conv3 = nn.Conv2d(16, 64, 3, padding=1) #8
#         self.bn2 = nn.BatchNorm2d(64)
#         # self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
#         self.pool2 = nn.MaxPool2d(2, 2) #16
#         self.drop2 = nn.Dropout(0.15)
#         self.conv5 = nn.Conv2d(64, 10, 3) #18
#         self.gap = nn.AdaptiveAvgPool2d((1,1))
#         # self.conv6 = nn.Conv2d(512, 1024, 3)
#         # self.conv7 = nn.Conv2d(1024, 10, 3)
#         # self.conv1 = nn.Conv2d(1, 32, 3, padding=1) 
#         # self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
#         # self.pool1 = nn.MaxPool2d(2, 2)
#         # self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
#         # self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
#         # self.pool2 = nn.MaxPool2d(2, 2)
#         # self.conv5 = nn.Conv2d(256, 512, 3)
#         # self.conv6 = nn.Conv2d(512, 1024, 3)
#         # self.conv7 = nn.Conv2d(1024, 10, 3)

#     def forward(self, x):
#         # x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
#         # x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
#         # x = F.relu(self.conv6(F.relu(self.conv5(x))))
#         # x = F.relu(self.conv7(x))

#         x = self.conv5(self.drop2(self.pool2(self.bn2(F.relu(self.conv3(F.relu(self.drop1(self.pool1(self.bn1(F.relu(self.conv1(x))))))))))))
#         # x = self.pool2(x)
#         # x = self.conv5(x)

#         # x = self.conv7(x)
#         # print(x.shape)

#         x = self.gap(x)
        
#         x = x.view(-1, 10)
#         return F.log_softmax(x)